# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

We can import the 'establishments.json' object as a database object by opening a new git terminal in the 'nosql-challenge' local github directory and running the following line:

    mongoimport "Resources\establishments.json" -d uk_food -c establishments --jsonArray --drop

In [11]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [12]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [13]:
# confirm that our new database was created
print(mongo.list_database_names())

['admin', 'config', 'local', 'uk_food']


In [14]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [15]:
# review the collections in our new database
print(db.list_collection_names())

['establishments']


In [16]:
# Preview a document in the establishments collection
pprint(db.establishments.find_one())

{'AddressLine1': 'The Bay',
 'AddressLine2': 'St Margarets Bay',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'Refreshment Kiosk',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4587.347174863443,
 'FHRSID': 254719,
 'LocalAuthorityBusinessID': 'PI/000069980',
 'LocalAuthorityCode': '182',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT15 6DY',
 'RatingDate': '2022-03-24T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('6602bad47e23cdf92282b695'),
 'geocode': {'latitude': '51.152225', 'longitude': '1.387974'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/254719',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          'extractDate': '0001-01-01T0

In [17]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [18]:
# Create a dictionary for the new restaurant data

new_restaurant = {
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
}

2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [19]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields

query = {'BusinessType': {'$regex' : 'Restaurant'}}
results = establishments.find_one(query)

business_keys = ['BusinessType', 'BusinessTypeID']
business_values = []

for each in business_keys:
    business_values.append(results[each])

business_values


['Restaurant/Cafe/Canteen', 1]

In [20]:
# Create a dictionary for the new restaurant data

new_restaurant_data = {'BusinessType' : business_values[0],
                       'BusinessTypeID' : business_values[1]}

new_restaurant_data

{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}

3. Update the new restaurant with the `BusinessTypeID` you found.

In [21]:
new_restaurant['BusinessType'] = new_restaurant_data['BusinessType']
new_restaurant['BusinessTypeID'] = new_restaurant_data['BusinessTypeID']


In [22]:
# Left in to delete new addition in case of code changes or to test code start to finish without adding duplicate entries into the db.

query = {'BusinessName': 'Penang Flavours'}
establishments.delete_many(query)

In [23]:
# Insert the new restaurant into the collection

establishments.insert_one(new_restaurant)

In [24]:
# Check/confirm that the new restaurant was inserted

query = {'BusinessName': 'Penang Flavours'}
results = establishments.find(query)
for result in results:
    pprint(result)

{'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'Distance': 4623.972328074718,
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': True,
 'Phone': '',
 'PostCode': 'SE18 7DY',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('6602baf3aa4e215fb18078a7'),
 'geocode': {'latitude': '51.49014200', 'longitude': '0.08384000'},
 'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}}


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [25]:
# Find how many documents have LocalAuthorityName as "Dover"
query = {'LocalAuthorityName': 'Dover'}
dover_documents = establishments.count_documents(query)


In [26]:
# Find how many documents have LocalAuthorityName as "Dover"
print(dover_documents)

994


In [27]:
# Delete all documents where LocalAuthorityName is "Dover"
establishments.delete_many(query)

In [28]:
# Check if any remaining documents include Dover
establishments.count_documents(query)

0

In [29]:
# Check that other documents remain with 'find_one'
establishments.find_one()

{'_id': ObjectId('6602bad47e23cdf92282b978'),
 'FHRSID': 1043695,
 'ChangesByServerID': 0,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'PostCode': 'CT19 6BL',
 'Phone': '',
 'RatingValue': '5',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingDate': '2018-04-04T00:00:00',
 'LocalAuthorityCode': '188',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'scores': {'Hygiene': 5, 'Structural': 5, 'ConfidenceInManagement': 5},
 'SchemeType': 'FHRS',
 'geocode': {'longitude': '1.195625', 'latitude': '51.083812'},
 'RightToReply': '',
 'Distance': 4591.765489457773,
 'NewRatingPending': False,
 'meta': {'dataSource': None,
  'extractDate': '0001-01-01

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [30]:
# Reference 2

# Change the data type from String to Decimal for longitude
field_to_convert = "geocode.longitude"

# Perform the update using update_many

establishments.update_many(
    {field_to_convert: {"$exists": True}},
    [{"$set": {field_to_convert: {"$toDouble": {"$convert": {"input": "$" + field_to_convert, "to": "decimal"}}}}}])

In [31]:
#Reference 2

# Change the data type from String to Decimal for latitude
field_to_convert = "geocode.latitude"

# Perform the update using update_many

establishments.update_many(
    {field_to_convert: {"$exists": True}},
    [{"$set": {field_to_convert: {"$toDouble": {"$convert": {"input": "$" + field_to_convert, "to": "decimal"}}}}}])

Use `update_many` to convert `RatingValue` to integer numbers.

In [32]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt", ""]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

In [33]:
# Reference 2

# Change the data type from String to Integer for RatingValue
establishments.update_many(
    {'RatingValue': {"$exists": True}},
    [{"$set": {"RatingValue": {"$toInt": {"$convert": {"input": "$RatingValue", "to": "int"}}}}}])

In [34]:
# Check that the coordinates and rating value are now numbers
rating_type = type(establishments.find_one()['RatingValue'])
lat_type = type(establishments.find_one()['geocode']['latitude'])
lon_type = type(establishments.find_one()['geocode']['longitude'])

print(f'RatingValue is now a {rating_type}')
print(f'Geocode Longitude is now a {lon_type}')
print(f'Geocode Latitude is now a {lat_type}')

RatingValue is now a <class 'int'>
Geocode Longitude is now a <class 'float'>
Geocode Latitude is now a <class 'float'>
